<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<p>
<h1>Algorísmica Avançada</h1>
<h2>Pràctica 2.A - Greedy </h2>
</center>
</p>
</div>

<div class="alert alert-info">
<center>
  <h1>Introducción</h1>
</center>

<p>Un algoritmo Greedy es aquel que trata de resolver un problema con la heurística de escoger, a cada paso, la opción óptima con la intención de encontrar una solución óptima global al problema. En caso general, este tipo de algoritmos no son capaces de encontrar la solución óptima, sin embargo, sí pueden encontrar soluciones subóptimas suficientemente cercanas con una coste computacional significativamente menor.</p>

<h3>Ejemplo</h3>
<p>Supongamos que intentamos encontrar la suma más grande en este grafo en forma de árbol. Un algoritmo greedy no consiste en encontrar una estrategia óptima global al problema, sino que a cada paso, buscará entre sus siguientes opciones cuál lleva más cerca de la solución (óptimo local).</p>
<img src="https://upload.wikimedia.org/wikipedia/commons/8/8c/Greedy-search-path-example.gif">

<h2>Estructura</h2>

Todos los algoritmos greedy comparten ciertas características:
<ol>
    <li><b>Conjunto</b> de elementos a partir de los cuales formar una solución.</li>
    <li>Criterio de <b>elección</b> del siguiente elemento candidato (Heurística).</li>
    <li>Criterio de <b>validación</b> sobre los elementos candidatos.</li>
    <li>Criterio de <b>terminación</b>, que indica cuando hemos alcanzado una solución completa.</li>
    <li>Métrica de <b>evaluación</b> de una solución, total o parcial.</li>
</ol>

<div class="alert alert-info">
<center>
  <h1>Contenido</h1>
  </center><p>



<div class="alert alert-success" style="width:90%; margin:0 auto;">
<h2>0 - Greedy Solver</h2>

Dado que todos los algoritmos Greedy siguen la misma estrategia, es posible crear una función que, dados los criterios de elección, validación y terminación, resuelva cualquier problema genéricamente. Adicionalmente, una función de evaluación de soluciones para determinar lo buena que es la solución propuesta por el algoritmo.

In [1]:
class GreedySolver:
    # Define los criterios que utilizará el GreedySolver
    def __init__(self, choice, is_valid, is_complete, evaluate,preprocess=lambda D: None):
        self._choice = choice
        self._is_valid = is_valid
        self._is_complete = is_complete
        self._evaluate = evaluate
        self._preprocess = preprocess 

    # Estrategia genérica de resolución de problemas
    def solve(self, D):
        self._preprocess(D)
        x = []
        # Mientras no se cumpla el criterio de terminación
        while D and not self._is_complete(x, D):
            # Escoge un elemento
            d = self._choice(x, D)
            # Comprueba si el elemento puede ser incluido en la solución
            if self._is_valid(x, d):
                x.append(d)
        # Devuelve la solución y la métrica
        return x, self._evaluate(x)

# Genera un conjunto de objetos de prueba
def random_objects(one_or_inf=True):
    objects = []
    # Genera una lista de listas, cada una de las cuales representa un elemento [peso,valor] o [peso,valor,cantidad]
    if one_or_inf:
        for p,v in zip(random.sample(range(1, 50), k=10), random.sample(range(1, 100), k=10)):
            objects.append([p,v])
    else:
        for p,v,c in zip(random.sample(range(1, 50), k=10), random.sample(range(1, 100), k=10),random.sample(range(1, 15), k=10)):
            objects.append([p,v,c])
    return objects

<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>1- El problema de la mochila</p></h2>
  
  <center><img src="img/knapsack.png" width=40%></center>
  
  <p>
    Nos encontramos en una habitación en la que hay $N$ objetos con pesos $w_1, w_2, w_3 ... w_N$ y tenemos una mochila que puede soportar una carga máxima de $W$.
    En esta práctica se pide que realicéis una implementación Greedy para resolver los siguientes problemas:
 <ol><li>
 **mochila:** En este caso buscamos llenar la mochila lo máximo posible. En otras palabras,queremos que $\sum_{i=0}^{N}{(w_i · b_i)}\leq W$ aproximándose lo más posible a $W$, donde $b_k$ es un valor binario que representa si el objeto es escogido o no.
 </li><li>
 **mochila_valor:** Igual que en el caso anterior, pero ahora cada objeto tendrá asignado un valor $v_1, v_2, v_3 ... v_N$. En este caso buscamos llenar la mochila maximizando el valor total de la mochila. Es decir queremos encontrar la combinación de objetos $b$ tal que $\arg_{b} \max{\sum_{i=0}^{N}{v_i · b_i}}$ manteniendo la condición anterior de que $\sum_{i=0}^{N}{(w_i · b_i)}\leq W$.
     </li>
     <li>
     **mochila_diversa:** Para este problema queremos meter en la mochila la mayor cantidad de objetos posibles. De esta manera, buscamos que la combinación de objetos $b$ tal que $\arg_{b} \max{\sum_{i=0}^{N}{b_i}}$.
     </li><li>
     **mochila_fraccionada:** Aquí podemos coger solo una parte del objeto. Así pues, dispondremos de una lista $x_1, x_2, x_3 ... x_N$ tal que $0 \leq x_i \leq 1$ y queremos que $\arg_{b} \max{\sum_{i=0}^{N}{v_i · x_i}}$ manteniendo la condición anterior de que $\sum_{i=0}^{N}{(w_i · x_i)}\leq W$.
     </li>
 </ol>
</p>

<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center>
<p>
**choice.** Esta función debe seleccionar el siguiente elemento.
    <ul>
        <li>Input:
            <ul>
                <li>x. Elementos que forman la solución actual</li>
                <li>D. Conjunto de elementos disponibles</li>
            </ul>
        </li>
        <li>Output:
            <ul>
                <li>d. Elemento seleccionado</li>
            </ul>
        </li>
        </ul>
</p>

<p>
**is_valid.** Esta función debe decidir si el elemento seleccionado puede añadirse a la solución.
    <ul>
        <li>Input:
            <ul>
                <li>x. Elementos que forman la solución actual</li>
                <li>d. Elemento seleccionado</li>
            </ul>
        </li>
        <li>Output:
            <ul>
                <li>valid. Booleano que indica la validez del elemento a añadir</li>
            </ul>
        </li>
    </ul>
</p>

<p>
**is_complete.** Esta función debe decidir si la solución encontrada es completa o si no puede continuarse.
    <ul>
        <li>Input:
            <ul>
                <li>x. Elementos que forman la solución actual</li>
                <li>D. Conjunto de elementos disponibles</li>
            </ul>
        </li>
        <li>Output:
            <ul>
                <li>complete. Booleano que indica si la solución es completa o imposible seguir</li>
            </ul>
        </li>
    </ul>
</p>

<p>
**evaluate.** Esta función devuelve una métrica indicando lo buena que es la solución encontrada.
    <ul>
        <li>Input:
            <ul>
                <li>x. Elementos que forman la solución</li>
            </ul>
        </li>
        <li>Output:
            <ul>
                <li>metric. Valor de la métrica evaluada sobre la solución</li>
            </ul>
        </li>
    </ul>
</p>
<p> <b> Nota: </b> El output de llamar a <i>GreedySolver.solve(D)</i> debe seguir un formato concreto:
    <ul>
        <li> La solución será la lista de objetos escogidos con la forma $[w_i,v_i]$ cuando solo se pueda coger un elemento de cada tipo y con la forma $[w_i,v_i,c_i]$ cuando la cantidad de objetos sea relevante.</li>
        <li> A la solución lo acompañará el resultado de la función <i>GreedySolver.evaluate()</i>: el peso total de la mochila $w_t$ o una tupla $(w_T,v_T)$ para indicar el peso y valor totales de la mochila (en los casos en que el valor sea relevante).</li>
    </ul>
</p>
</div>


Define para cada una de las variantes del problema de la mochila las funciones necesarias para el correcto funcionamiento del GreedySolver.

In [2]:
# Capacidad de la mochila
W = 150

In [3]:
def choice1(x, D):
    D.sort(reverse = True,key = crit) #Decidim el criteri
    d = D[0] #La solucio final es el pes de D 
    D.remove(D[0]) #Elimina D[0] O(n)
    return d
    
def crit(x):
    return x[0] #Pes

def is_valid1(x, d):
    valid = False #Posem a false
    weight = 0 #Posem una variable a 0
    for n in x: #Recorrem tota la solucio actual O(1)
        weight = weight + n[0] #Li sumem a la nostre variable el pes n[0]
    if(len(x)==0): #Si la longitud de x es 0 es vàlid O(n) O(1)
        valid =  True
    else:
        if(d[0]<= weight): #Si el pes de la solució final es menor o igual a la nostre variable retorna True O(n)
            valid = True
        else: #Sino False
            valid = False
    return valid

def is_complete1(x, D):
    weight = 0 #Variable a 0
    complete = False #Posem compelte a False
    for n in x: #Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        weight = weight+n[0] 
    if weight < W: #Si el pes es menor a W O(n)
        weight = weight+min(D[0]) #Li sumem a la variable el min(D[0]) O(n)
        if(weight < W): #Si el pes es menor a W return False
            complete = False
        elif weight >= W: #Si el pes es major o igual retornem True O(n)
            complete = True
    return complete

def evaluate1(x):
    weight = 0
    for n in x: #Recorrem la solucio actual i retornem el pes O(1)
        weight += n[0]
    return weight

In [4]:
def choice2(x, D):
    D.sort(reverse = True,key = crit2) #Decidim el criteri
    d = D[0] #La solucio final es el pes de D
    D.remove(D[0]) #Eliminem D[0] de D O(n)
    return d
    
def crit2(x):
    return x[1]/x[0] #Proporcio, valor entre pes

def is_valid2(x, d):
    valid = False
    weight = 0
    for n in x: #Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        weight = weight + n[0]
    if(len(x)==0): #Si la longitud de x es 0 True O(1)
        valid =  True
    else: 
        if(d[0]<= (W-weight)): #Si el pes de la solucio es menor o iguall a W-la nostre variable es True O(n)
            valid = True
        else:#Si no retorna False
            valid = False
    return valid

def is_complete2(x, D):
    weight = 0
    complete = False
    for n in x:#Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        weight = weight+n[0]
    if weight < W: #Si la nostre variable es menor a 150 O(n)
        weight = weight+min(D[0]) #Li sumem el mind(D[0]) O(n)
        if(weight < W):#Si la nostre variable es menor a 150 O(n)
            complete = False #False
        elif weight >= W: #Si es major o igual a 150 true O(n)
            complete = True
    return complete

def evaluate2(x):
    value = 0
    for n in x: #Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        value += n[0]
    return value

In [5]:
def choice3(x, D):
    D.sort(reverse = False,key = crit3) #Decidim el criteri
    d = D[0] #La solucio final es el pes de D
    D.remove(D[0]) #Eliminem D[0] de D O(n)
    return d
    
def crit3(x):
    return x[0] #Pes

def is_valid3(x, d):
    valid = False #Inicialitzem a false
    weight = 0 #Inicialitzem a 0
    for n in x: #Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        weight = weight + n[0]
    if(len(x)==0): #Si la longitud de la solucio actual es 0 True O(n)
        valid =  True
    else:
        if(d[0]<= (W-weight)): #Si el pes de la solucio es menor o iguall a W-la nostre variable es True O(n)
            valid = True
        else: #Si no retorna False
            valid = False
    return valid

def is_complete3(x, D):
    weight = 0
    complete = False
    for n in x: #Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        weight = weight+n[0] 
    if weight < W: #Si la nostre variable es menor a 150 O(n)
        weight = weight+min(D[0]) #Li sumem el mind(D[0])
        if(weight < W): #Si la nostre variable es menor a 150 False O(n)
            complete = False
        elif weight >= W: #Si es major o igual a 150 true O(n)
            complete = True
    return complete

def evaluate3(x):
    value = 0
    for n in x:#Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        value += n[0]
    return value

In [6]:
def choice4(x, D):
    D.sort(reverse = True,key = crit4) #Decidim el criteri
    d = D[0] #La solucio final es el pes de D
    D.remove(D[0]) #Eliminem D[0] de D O(n)
    return d
    
def crit4(x):
    return x[1]/x[0] #Proporcio, valor entre pes

def is_valid4(x, d):
    weight = 0
    valid = False
    for n in x: #Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        weight = weight + n[0]
    if(weight == W): # Si el pes es igual a W False O(n)
        valid = False
    else:
        if((weight + d[0] <= W)): #Si la suma del pes amb el pes de la solucio fianl es menor o igual a 150 True O(n)
            valid = True
        else: #Si es el contrari llavors fem la proporcio,  O(n)
            frac = W - weight #Fraccio es 150 - la nostre variable
            d[1] = (d[1]*frac)/d[0] #El valor de la solucio final es = el valor per frac entre el pes de la solucio O(n)
            d[0]  = frac #Fem que d[0] sigui frac
            valid = True #True
    return valid
        

def is_complete4(x, D):
    complete = False
    weight = 0
    for n in x: #Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        weight = weight + n[0]
    if (weight == W): # Si el pes es igual a W true O(n)
        complete = True
    else: #False O(n)
        complete = False

def evaluate4(x):
    value = 0
    for n in x: #Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] O(1)
        value += n[0]
    return value

Añade aquí el código para instanciar un GreedySolver y utilizarlo para resolver cada uno de los problemas y mostrar la solución y su evaluación en pantalla.

In [7]:
import copy
import random
# Con un seed fijo, el generador de objetos no cambia
random.seed(0)

# Genera una lista de elementos al azar en forma listas [peso, valor]
D = random_objects()
print(D)
# Instancia los GreedySolvers y úsalos para resolver las variantes del problema de la mochila
G1 = GreedySolver(choice1,is_valid1,is_complete1,evaluate1)
print(G1.solve(copy.deepcopy(D)))
G2 = GreedySolver(choice2,is_valid2,is_complete2,evaluate2)
print(G2.solve(copy.deepcopy(D)))
G3 = GreedySolver(choice3,is_valid3,is_complete3,evaluate3)
print(G3.solve(copy.deepcopy(D)))
G4 = GreedySolver(choice4,is_valid4,is_complete4,evaluate4)
print(G4.solve(copy.deepcopy(D)))

[[25, 75], [27, 28], [3, 65], [17, 18], [33, 37], [32, 97], [26, 13], [20, 80], [31, 33], [23, 69]]
([[33, 37], [32, 97], [31, 33], [27, 28], [26, 13]], 149)
([[3, 65], [20, 80], [32, 97], [25, 75], [23, 69], [33, 37]], 136)
([[3, 65], [17, 18], [20, 80], [23, 69], [25, 75], [26, 13], [27, 28]], 141)
([[3, 65], [20, 80], [32, 97], [25, 75], [23, 69], [33, 37], [14, 14.903225806451612]], 150)


<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h3>1.1- La mochila con varios objetos</h3>
  <p>Supongamos ahora que en lugar de tener un único elemento de cada objeto, tenemos una cantidad determinada de cada uno y la solución no solo consiste en coger el objeto o no, sino también en cuántos cogemos de cada objeto. ¿Cómo cambia la implementación de la solución del ejercicio 2? Impleméntalo. ¿Es posible resolverlo con el esquema dado del GreedySolver? En caso negativo, crea una nueva clase GreedySolver_v con los cambios que consideres oportunos.
</p>

In [8]:
def choice2_v(x, D):
    D.sort(reverse = True,key = crit5) #Definiim el criteri
    d = D[0]
    n = evaluate2_v(x) #fem que ens cridi recursivament el metode evaluate
    cant = (W - n)//d[2] #Cant es 150 - el valor de evaluate entre la quantitat de la solucio final
    if cant < d[2]: #S i cant es menor a la quantitat la igualem O(n)
          d[2] = cant
    D.remove(D[0]) # Eliminem D[0] de D O(n)
    return d

    
def crit5(x):
    return x[1]/x[0] #proporcio entre valor i pes

def is_valid2_v(x, d):
    weight = evaluate2_v(x) #Cridem al metode evaluate
    valid = False
    if(len(x) == 0): #Si la longitud de la solucio actual es 0 True O(1)
        valid = True
    else:
        if(d[2] == 0): # Si la quantitat es 0 llavors False O(n)
            valid = False
        elif(d[0]*d[2] <= (W-weight)): #Si la quantitat per el pes es menor a 150 - el pes llavors True O(n)
            valid = True
        else:  #False O(n)
            valid = False
    return valid
        

def is_complete2_v(x, D):
    complete = False
    weight = evaluate2_v(x) #Cridem al metode evaluate
    if weight < W: #Si el pes es menor a 150 O(n)
        weight = weight + min(D)[0] #Li sumem el mind(D[0])
        if (weight <= W): #Si el pes es menor o igual a 150 llavors false O(n)
            complete = False
        else: #True O(n)
            compelte = True
    else: #O(n)
        complete = True
    
def evaluate2_v(x):
    value = 0
    for n in x:#Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] i li multipliquem la quantitat O(1)
        value += n[0]*n[2]
    return value

In [9]:
import random
random.seed(0)

# Genera una lista de elementos al azar en forma de listas [peso, valor, cantidad]
D=random_objects(False)
print(D)
# Instancia el GreedySolver y úsalo para resolver esta variante del problema de la mochila
G2_v = GreedySolver(choice2_v,is_valid2_v,is_complete2_v,evaluate2_v)
print(G2_v.solve(copy.deepcopy(D)))

[[25, 75, 12], [27, 28, 13], [3, 65, 10], [17, 18, 3], [33, 37, 5], [32, 97, 2], [26, 13, 9], [20, 80, 7], [31, 33, 6], [23, 69, 11]]
([[3, 65, 10], [32, 97, 2], [17, 18, 3]], 145)


<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h3>1.2- La mochila con infinitos objetos</h3>
<p>Supongamos ahora que tenemos una cantidad ilimitada de cada objeto. ¿Cómo cambia la implementación de la solución del ejercicio ahora? Impleméntalo. ¿Es posible resolverlo con el esquema dado del GreedySolver? En caso negativo, crea una nueva clase GreedySolver_inf con los cambios que consideres oportunos.
</p>

In [10]:
def choice2_inf(x, D):
    D.sort(reverse = True,key = crit6) #Definim el criteri
    d = D[0] #La solucio final es igual a D[0]
    n = evaluate2_inf(x) #Cridem a evaluate
    u = W//d[0] # 150 entre el pes de la solucio final
    d.append(u) #Afegim u a la solucio final O(1)
    D.remove(D[0]) #O(n)
    return d
    
def crit6(x):
    return x[1]/x[0] #Proporcio entre el valor i el pes


def is_valid2_inf(x, d):
    weight = evaluate2_inf(x)
    valid = False
    if(len(x) == 0): #Si la longitud de x es 0 llavors true O(1)
        valid = True
    else: #O(n)
        if( d[2] == 0): #Si la quantitat es igual a 0 retorna False O(n)
            valid = False
        elif(d[0]*d[2] <= (W-weight)): #Si la multiplicacio entre quantitat i pes es menor a 150 - pes retorna true O(n)
            valid = True
        else: #False O(n)
            valid = False
    return valid

def is_complete2_inf(x, D):
    complete = False
    weight = evaluate2_inf(x) #Cridem a evaluate
    if weight < W: #Si el pes menor a 150 O(n)
        weight = weight + min(D)[0] #Li sumem el mind(D[0]) O(n)
        if (weight <= W): #Si el pes es menor o igual a 150 llavors false O(n)
            complete = False
        else: #True O(n)
            complete = True
    else:  #true O(n)
        complete = True


def evaluate2_inf(x):
    value = 0
    for n in x:#Recorrem la solucio actual i li sumem a la nostre variable el pes n[0] i li multipliquem W//n[0] O(1)
        value = value + n[0]*(W//n[0])
    return value

In [11]:
import random
random.seed(0)

# Genera una lista de elementos al azar en forma listas [peso, valor]
D = random_objects()
print(D)
# Instancia el GreedySolver y úsalo para resolver esta variante del problema de la mochila

G2_inf = GreedySolver(choice2_inf,is_valid2_inf,is_complete2_inf,evaluate2_inf)
print(G2_inf.solve(copy.deepcopy(D)))

[[25, 75], [27, 28], [3, 65], [17, 18], [33, 37], [32, 97], [26, 13], [20, 80], [31, 33], [23, 69]]
([[3, 65, 50]], 150)


<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Cuestiones</h3></p> </center> </div>

<ul>
    <li>¿En qué casos se encuentra solución óptima al problema?</li>
    <li>Explica las soluciones planteadas y analiza su complejidad.</li>
    <li>Explica la estrategia que has empleado para las variantes del problema 2, enfatizando las diferencias principales con respecto al problema original ¿Ha cambiado la complejidad? Justifica tu respuesta.</li>
</ul>

__Escribe aquí tus respuestas__

<h4> Pregunta 1: En Greedy en teoria nunca hay una solucion optima no obstante,la solucion que creo que es optima para este problema se encuentra en esos casos en los que se llega a 150 o casi. Por ejemplo en el caso 1 de la mochila con peso,en la mochila fraccioanria y por ultimo en la mochila de objetos infinitos</h4> 

<h4> Pregunta 2: Las soluciones planteadas están explicadas en el código junto con su complejidad </h4> 

<h4> Pregunta 3: La estrategia en las variantes del problema 2 han sido ir llamando de forma recursiva la funcion evaluate para que nos de el valor en cuestión, a partir de ese valor hemos tenido que ir modificando sobretodo las partes de is_valid y is_complete porque no son las mismas que para los casos anteriores, ya que en estos 2 casos, la mochila tiene objetos con una cantidad asociada y por lo tanto se tiene que ver cuantos objetos se cojen de un mismo tipo y en el otro problema nos pide hacer una mochila con infinitos objetos. Por lo tanto hemos tenido que modificar el codigo para satisfacer estos dos enunciados de problemas</h4>

<h4> Pregunta 4: La complejidad obviamente ha cambiado, ya que el código al final ha tenido que cambiar para poder representar estas variantes del problema. Se ha hecho de manera recursiva por lo tanto la complejidad ha mejorado y además es más eficiente que los primeros problemas </h4>

<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>
<p>
La entrega de esta práctica (junto a la de programación dinámica) se podrá realizar en el campus virtual hasta el día __25 de Noviembre a las 23:55__. En la tarea que se habilitará en el campus deberéis colgar __un archivo .zip__ que contenga __únicamente__ los notebooks de las dos prácticas siguiendo la siguiente nomenclatura:
</p>
<p>
```
*AA_P2_<apellido1>_<apellido2>_<nombre>.zip
|___* AA_Greedy_<apellido1>_<apellido2>_<nombre>.ipynb
|___* AA_PD_<apellido1>_<apellido2>_<nombre>.ipynb
```

</p>
<p>
Por ejemplo:
</p>
<p>
```
AA_P2_Doe_Ritchie_John.zip
```

</p>
<p>

Es fundamental que el código esté bién comentado y con un análisis de complejidad exhaustivo del algoritmo.También resulta de gran importancia que la entrega siga el formato indicado. 
</p>
</div>